In [6]:
import tensorflow as tf
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd
import pickle
import random

Using TensorFlow backend.


In [7]:
import json
with open('intents edited.json') as file:
    intents = json.load(file)

In [8]:
data = pickle.load( open( "HC-assistant-data.pkl", "rb" ) )
words = data['words']
classes = data['classes']

In [9]:
with open(f'HC-assistant-model.pkl', 'rb') as f:
    model = pickle.load(f)

In [10]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                

    return(np.array(bag))

In [7]:
def chat():
    print("Chat with HEIGHTS COMMUNICATIONS bot assistant! (type quit to exit)\n\n")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
                
        input_data = pd.DataFrame([bow(inp, words)], dtype=float, index=['input'])
        results = model.predict([input_data])[0]
        results_index = np.argmax(results)
        tag = classes[results_index]
        
        if results[results_index] > 0.7:
            for tg in intents["intents"]:
                if tg['tag'] == tag:
                    responses = tg["responses"]
            print("Assistant:", random.choice(responses))
        else:
            print("Assistant:","I am sorry. I am not aware of what you are asking. Please call us at +923232779999")
chat()

Chat with HEIGHTS COMMUNICATIONS bot assistant! (type quit to exit)


You: Hi
Assistant: Hello!
You: Can you tell me about your business?
Assistant: We, Heights Communication is a BTL agency based in Pakistan and UAE. We do indoor and outdoor advertising campaigns, marketing campaigns and branding campaign for our clients. For more information about our work, you can call us at +923232779999
You: That's nice. What are your recent projects?
Assistant: To see our recent project, please click on  ' our projects' tab. For more information about our past projects call us at +923232779999
You: Alright.Can you tell me about your services? 
Assistant: We provide variety of services including corporate launch, activities, outdoor promotions, digital branding, product display, digital printing, hoarding advertising and much more. For more information call us at +923232779999
You: Interesting. What are your prices?
Assistant: For rates and pricing of a particular service, call us now at +92323277